# auto-sklearn

#### Author's description:

auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator

#### Useful links:

[install link](https://automl.github.io/auto-sklearn/master/installation.html),
[git](https://github.com/automl/auto-sklearn),
[manual](https://automl.github.io/auto-sklearn/master/manual.html),
[parallel instances](https://automl.github.io/auto-sklearn/master/examples/example_parallel_manual_spawning.html),
[parallel runs on one machine](https://automl.github.io/auto-sklearn/master/examples/example_parallel_n_jobs.html),
[cross validation](https://automl.github.io/auto-sklearn/master/examples/example_crossvalidation.html),
[feature types](https://automl.github.io/auto-sklearn/master/examples/example_feature_types.html)

#### Usage Note

auto-sklearn builds ensembles, provides good control of the auto-ML run details, and makes exporting easy due to its scikit-learn foundations.

## Install and import

In [1]:
!sudo pip install auto-sklearn==0.6.0

     |████████████████████████████████| 3.9 MB 5.4 MB/s 
     |████████████████████████████████| 964 kB 76.7 MB/s 
     |████████████████████████████████| 293 kB 78.8 MB/s 
     |████████████████████████████████| 94 kB 71.9 MB/s 
     |████████████████████████████████| 10.8 MB 96 kB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.6.0-cp36-cp36m-linux_x86_64.whl size=4268838 sha256=122dbf31bf4833404cec70f14118fa45d2c5c0e8c366e6889d576a0adfd3fd10
  Stored in directory: /tmp/pip-ephem-wheel-cache-2it71xe2/wheels/1b/9a/78/0dd70afe14ccb1f8d3fb51cedda41b52c4cfc398eeb3efeb60
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13335 sha256=f63d40d68154525423d8304f7d30a5267f4684fc114b91def3694c6e341cf46d
  Stored in directory: /tmp/pip-ephem-wheel-cache-2it71xe2/wheels/ba/2a/e1/6f7be2e2ea150e2486bff64fd6f0670f4f35f4c8f31c819fb8
  Created wheel for ConfigSpace: filename=ConfigSpace-0.4.13-cp36-cp36m-linux_x86_64.whl size=3102514 sha256=e3353bd6405dab0f8ae8b

In [2]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import numpy as np
import subprocess

In [3]:
#the latest verified version of autosklearn in this notebook was 0.5.2
autosklearn.__version__

'0.6.0'

In [4]:
#there can be a lot of warnings in auto-sklearn
#especially if you overwrite existing files
#turning off for demo purposes

import warnings
warnings.filterwarnings("ignore")

## Take a look at the classification function

auto-sklearn is mostly a wrapper around scikit-learn. It was not the intention of the authors to allow user control over details such as the modeling algorithm and typical hyper-parameter choices. Control is several layers deep in the [SMAC](https://automl.github.io/SMAC3/master/) space and scenario settings. The user can control the time is takes to build the ensemble, the resampling strategy and the parallelization of the work across CPUs on the machine. These will be demonstrated below.

In [5]:
?autosklearn.classification.AutoSklearnClassifier

Init signature:
autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    per_run_time_limit=360,
    initial_configurations_via_metalearning=25,
    ensemble_size:int=50,
    ensemble_nbest=50,
    ensemble_memory_limit=1024,
    seed=1,
    ml_memory_limit=3072,
    include_estimators=None,
    exclude_estimators=None,
    include_preprocessors=None,
    exclude_preprocessors=None,
    resampling_strategy='holdout',
    resampling_strategy_arguments=None,
    tmp_folder=None,
    output_folder=None,
    delete_tmp_folder_after_terminate=True,
    delete_output_folder_after_terminate=True,
    shared_mode=False,
    n_jobs:Union[int, NoneType]=None,
    disable_evaluator_output=False,
    get_smac_object_callback=None,
    smac_scenario_args=None,
    logging_config=None,
    metadata_directory=None,
)
Docstring:      This class implements the classification task.
Init docstring:
Parameters
----------
time_left_for_this_task : int, optional (default=3600)

## Heart Disease

#### Load the heart disease dataset

Note that in this cell we are calling **sklearn.model_selection.train_test_split()** twice and creating two sets of heart disease (hd) data for model fitting and testing. One is for the hd data without one hot encoding (ohe) and the other has the ohe columns. 

auto-sklearn accepts a list of categorical features and has several methods for treating categorical data. In this notebook we try both approaches - building ohe columns ourselves and letting auto-sklearn do its thing.

In [6]:
'''
/mnt/data/raw/heart.csv

attribute documentation:
      age: age in years
      sex: sex (1 = male; 0 = female)
      cp: chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
     trestbps: resting blood pressure (in mm Hg on admission to the 
        hospital)
     chol: serum cholestoral in mg/dl
     fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
     restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
     thalach: maximum heart rate achieved
     exang: exercise induced angina (1 = yes; 0 = no)
     oldpeak = ST depression induced by exercise relative to rest
     slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
     ca: number of major vessels (0-3) colored by flourosopy
     thal: 
         3 = normal; 
         6 = fixed defect; 
         7 = reversable defect
     target: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
 '''

#load and clean the data----------------------

#column names
names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang', \
         'oldpeak','slope','ca','thal','target']

#load data from Domino project directory
hd_data = pd.read_csv("../data/raw/heart.csv", header=None, names=names)

#in case some data comes in as string
#convert to numeric and coerce errors to NaN
for col in hd_data.columns:  # Iterate over chosen columns
    hd_data[col] = pd.to_numeric(hd_data[col], errors='coerce')
    
#drop nulls
hd_data.dropna(inplace=True)

#non-ohe data---------------------------------
   
#load the X and y set as a numpy array
X_hd = hd_data.drop('target', axis=1).values
y_hd = hd_data['target'].values

#build the train and test sets
X_hd_train, X_hd_test, y_hd_train, y_hd_test = \
    sklearn.model_selection.train_test_split(X_hd, y_hd, random_state=12)

#now do ohe-----------------------------------

#function to do one hot encoding for categorical columns
def create_dummies(data, cols, drop1st=True):
    for c in cols:
        dummies_df = pd.get_dummies(data[c], prefix=c, drop_first=drop1st)  
        data=pd.concat([data, dummies_df], axis=1)
        data = data.drop([c], axis=1)
    return data

cat_cols = ['cp', 'restecg', 'slope', 'ca', 'thal']
hd_data = create_dummies(hd_data, cat_cols)
    
#load the X and y set as a numpy array
X_hd_ohe = hd_data.drop('target', axis=1).values
y_hd_ohe = hd_data['target'].values

#build the train and test sets
X_hd_ohe_train, X_hd_ohe_test, y_hd_ohe_train, y_hd_ohe_test = \
    sklearn.model_selection.train_test_split(X_hd_ohe, y_hd_ohe, \
                                             random_state=12)

#### Function to delete the output and temp directories of auto-sklearn

You'll need to clear the previous folders to avoid overwrite errors. Alternatively, you can create new output directories.

In [7]:
def cleanup(directories_, delete_):
    for d in directories_:
        if delete_:
            print('deleting', d)
            completed = subprocess.run(
                ['rm', '-rf', d],
                stdout=subprocess.PIPE,
            )
            print(completed.stdout.decode('utf-8'))

#### Build a model on ohe data with holdout

In [8]:
%%time

#set and clear the output directories
directories = ['../results/tmp_hd_holdout', \
               '../results/out_hd_holdout']
cleanup(directories, True)

#build the auto-sklearn routine
automl_hd_ohe = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories[0],
    output_folder=directories[1],
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67}
)

#call it
automl_hd_ohe.fit(X_hd_ohe_train, y_hd_ohe_train, \
                  dataset_name='heart_disease')

#save the predicitons
predictions_hd_ohe = automl_hd_ohe.predict(X_hd_ohe_test)

deleting ../results/tmp_hd_holdout

deleting ../results/out_hd_holdout

[WARNING] [2020-08-27 21:04:06,349:EnsembleBuilder(1):heart_disease] No models better than random - using Dummy Score!
[WARNING] [2020-08-27 21:04:06,367:EnsembleBuilder(1):heart_disease] No models better than random - using Dummy Score!
CPU times: user 7.52 s, sys: 268 ms, total: 7.79 s
Wall time: 55.8 s


#### Fitting with autosklearn

A common mistake is to call **fit_ensemble()** after already running **fit()**. **fit()** both optimizes the machine learning models and builds an ensemble out of them. To disable ensembling when running **fit()** (with parallel instances for example) set ensemble_size to 0. Then **fit_ensemble()** would be needed once all models have been built.

To save fitted models, use typical [pickle procedures](https://scikit-learn.org/stable/modules/model_persistence.html#persistence-example).

#### Metrics

Accuracy, sprint stats, and model details are available. 

Later we will run auto-sklearn in parallel. Note the number of models built here and compare it to the number built with parallelization turned on. 

The model details give you insight into what auto-sklearn is doing under the hood. You can see the modeling algorithm used and all the parameter settings. 

In [10]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ohe.sprint_statistics())
print(' ')
print('-----------------------------------------')
print(' ')
print('Model Details:')
print(automl_hd_ohe.show_models())

Accuracy:
0.75
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.866667
  Number of target algorithm runs: 18
  Number of successful target algorithm runs: 18
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

 
-----------------------------------------
 
Model Details:
[(0.160000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'no_encoding', 'classifier:__choice__': 'extra_trees', 'imputation:strategy': 'median', 'preprocessor:__choice__': 'feature_agglomeration', 'rescaling:__choice__': 'normalize', 'classifier:extra_trees:bootstrap': 'True', 'classifier:extra_trees:criterion': 'entropy', 'classifier:extra_trees:max_depth': 'None', 'classifier:extra_trees:max_features': 0.9189357795140339, 'classifier:extra

#### Do the same thing (build a model on ohe data with holdout) but this time with parallelization turned on

In [11]:
%%time

#set and clear the output directories
directories_parallel = ['../results/tmp_hd_holdout_parallel', \
                        '../results/out_hd_holdout_parallel']
cleanup(directories_parallel, True)

#build the auto-sklearn routine
automl_hd_ohe_p = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories_parallel[0],
    output_folder=directories_parallel[1],
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    
    #turn on parallelization
    n_jobs=4,
    seed=5,
    
    delete_output_folder_after_terminate=False,
    delete_tmp_folder_after_terminate=False,
)

#call it
automl_hd_ohe_p.fit(X_hd_ohe_train, y_hd_ohe_train, \
                    dataset_name='heart_disease')

#save the predicitons
predictions_hd_ohe_p = automl_hd_ohe_p.predict(X_hd_ohe_test)

deleting ../results/tmp_hd_holdout_parallel

deleting ../results/out_hd_holdout_parallel

[WARNING] [2020-08-27 21:06:32,372:EnsembleBuilder(236996814):heart_disease] No models better than random - using Dummy Score!
[WARNING] [2020-08-27 21:06:32,378:EnsembleBuilder(236996814):heart_disease] No models better than random - using Dummy Score!
CPU times: user 3.68 s, sys: 300 ms, total: 3.98 s
Wall time: 57.1 s


In [12]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe_p))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ohe_p.sprint_statistics())

Accuracy:
0.7368421052631579
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 25
  Number of successful target algorithm runs: 21
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



In [13]:
print('Model Details:')
print(automl_hd_ohe_p.show_models())

Model Details:
[(0.220000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'qda', 'imputation:strategy': 'most_frequent', 'preprocessor:__choice__': 'fast_ica', 'rescaling:__choice__': 'minmax', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'False', 'classifier:qda:reg_param': 0.8791968590596918, 'preprocessor:fast_ica:algorithm': 'parallel', 'preprocessor:fast_ica:fun': 'cube', 'preprocessor:fast_ica:whiten': 'False'},
dataset_properties={
  'task': 1,
  'sparse': False,
  'multilabel': False,
  'multiclass': False,
  'target_type': 'classification',
  'signed': False})),
(0.160000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'multinomial_nb', 'imputation:strategy': 'mean', 'preprocessor:__choice__': 'liblinear_svc_preprocessor', 'rescaling:__choice__': 'minmax', 'categorica

#### Try with feat_type option instead of ohe (still using parallel and holdout)

In [14]:
# %%time

# #set and clear the output directories
# directories_parallel_ft = ['../results/tmp_hd_holdout_parallel_ft', \
#                            '../results/out_hd_holdout_parallel_ft']
# cleanup(directories_parallel_ft, True)

# #build the auto-sklearn routine
# automl_hd_ft_p = autosklearn.classification.AutoSklearnClassifier(
#     time_left_for_this_task=60,
#     per_run_time_limit=30,
#     tmp_folder=directories_parallel_ft[0],
#     output_folder=directories_parallel_ft[1],
#     disable_evaluator_output=False,
#     # 'holdout' with 'train_size'=0.67 is the default argument setting
#     # for AutoSklearnClassifier. It is explicitly specified in this example
#     # for demonstrational purpose.
#     resampling_strategy='holdout',
#     resampling_strategy_arguments={'train_size': 0.67},
#     n_jobs=4,
#     seed=5,
#     delete_output_folder_after_terminate=False,
#     delete_tmp_folder_after_terminate=False,
# )

# feat_type = ['Numerical','Numerical','Categorical','Numerical',\
#              'Numerical', 'Numerical','Categorical', 'Numerical',\
#              'Numerical','Numerical', 'Categorical','Numerical',\
#              'Categorical']

# #call it
# automl_hd_ft_p.fit(X_hd_train, y_hd_train, \
#                    dataset_name='heart_disease', feat_type=feat_type)

# #save the predicitons
# predictions_hd_ft_p = automl_hd_ft_p.predict(X_hd_test)

In [15]:
# print('Accuracy:')
# print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
#                                      predictions_hd_ft_p))
# print(' ')
# print('-----------------------------------------')
# print(' ')
# print('Sprint Stats:')
# print(automl_hd_ft_p.sprint_statistics())

In [16]:
# print('Model Details:')
# print(automl_hd_ft_p.show_models())

#### Try with CV instead of Holdout (using ohe and parallel)

CV requires an extra step to fit our ensemble on all the data. During **fit()**, models are fit on individual cross-validation folds. To use all available data, we call **refit()** which trains all models in the final ensemble on the whole dataset. Also, when using CV, **fit()** changes the data in place, but refit needs the original data. So we use the **copy()** function. In practice, you might want to reload the data.

In [17]:
# %%time

# #set and clear the output directories
# directories_parallel_cv = ['../results/tmp_hd_holdout_parallel_cv', \
#                            '../results/out_hd_holdout_parallel_cv']
# cleanup(directories_parallel_cv, True)

# #build the auto-sklearn routine
# automl_hd_cv_p = autosklearn.classification.AutoSklearnClassifier(
#     time_left_for_this_task=60,
#     per_run_time_limit=30,
#     tmp_folder=directories_parallel_cv[0],
#     output_folder=directories_parallel_cv[1],
#     disable_evaluator_output=False,
#     # 'holdout' with 'train_size'=0.67 is the default argument setting
#     # for AutoSklearnClassifier. It is explicitly specified in this example
#     # for demonstrational purpose.
#     resampling_strategy='cv',
#     resampling_strategy_arguments={'folds': 5},
#     n_jobs=4,
#     seed=5,
#     delete_output_folder_after_terminate=False,
#     delete_tmp_folder_after_terminate=False,
# )

# #call it
# automl_hd_cv_p.fit(X_hd_ohe_train.copy(), y_hd_ohe_train.copy(), \
#                    dataset_name='heart_disease')
# automl_hd_cv_p.refit(X_hd_ohe_train.copy(), y_hd_ohe_train.copy())

# #save the predicitons
# predictions_hd_cv_p = automl_hd_cv_p.predict(X_hd_ohe_test)

In [18]:
# print('Accuracy:')
# print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
#                                      predictions_hd_cv_p))
# print(' ')
# print('-----------------------------------------')
# print(' ')
# print('Sprint Stats:')
# print(automl_hd_cv_p.sprint_statistics())

In [19]:
# print('Model Details:')
# print(automl_hd_cv_p.show_models())

## Breast Cancer

#### Load the breast cancer data

In [20]:
from sklearn.datasets import load_breast_cancer
print(sklearn.datasets.load_breast_cancer()['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [21]:
#load from sklearn
X_bc, y_bc = sklearn.datasets.load_breast_cancer(return_X_y=True)

#build the train and test sets
X_bc_train, X_bc_test, y_bc_train, y_bc_test = \
    sklearn.model_selection.train_test_split(X_bc, y_bc, random_state=1)

#### Build a model using holdout and parallelization

In [22]:
%%time

#set and clear the output directorie
directories_bc = ['../results/tmp_bc', '../results/out_bc']
cleanup(directories_bc, True)

#build the auto-sklearn routine
automl_bc = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories_bc[0],
    output_folder=directories_bc[1],
    disable_evaluator_output=False,
    # 'holdout' with 'train_size'=0.67 is the default argument setting
    # for AutoSklearnClassifier. It is explicitly specified in this example
    # for demonstrational purpose.
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    n_jobs=4,
    seed=5,
    delete_output_folder_after_terminate=False,
    delete_tmp_folder_after_terminate=False,
)

#call it
automl_bc.fit(X_bc_train, y_bc_train, dataset_name='breast_cancer')

#save the predicitons
predictions_bc = automl_bc.predict(X_bc_test)

deleting ../results/tmp_bc

deleting ../results/out_bc

[WARNING] [2020-08-27 21:08:34,667:EnsembleBuilder(236996814):breast_cancer] No models better than random - using Dummy Score!
[WARNING] [2020-08-27 21:08:34,675:EnsembleBuilder(236996814):breast_cancer] No models better than random - using Dummy Score!
CPU times: user 2.92 s, sys: 188 ms, total: 3.11 s
Wall time: 56.8 s


In [23]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_bc_test, \
                                     predictions_bc))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_bc.sprint_statistics())

Accuracy:
0.951048951048951
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: breast_cancer
  Metric: accuracy
  Best validation score: 0.971631
  Number of target algorithm runs: 21
  Number of successful target algorithm runs: 18
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0



In [24]:
print('Model Details:')
print(automl_bc.show_models())

Model Details:
[(0.220000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'no_encoding', 'classifier:__choice__': 'sgd', 'imputation:strategy': 'median', 'preprocessor:__choice__': 'pca', 'rescaling:__choice__': 'none', 'classifier:sgd:alpha': 2.2001685667477848e-05, 'classifier:sgd:average': 'False', 'classifier:sgd:fit_intercept': 'True', 'classifier:sgd:learning_rate': 'invscaling', 'classifier:sgd:loss': 'perceptron', 'classifier:sgd:penalty': 'elasticnet', 'classifier:sgd:tol': 0.0003374824245442477, 'preprocessor:pca:keep_variance': 0.7649896992433587, 'preprocessor:pca:whiten': 'False', 'classifier:sgd:eta0': 3.1030969765650854e-06, 'classifier:sgd:l1_ratio': 5.837966378962064e-05, 'classifier:sgd:power_t': 0.20397285210565702},
dataset_properties={
  'task': 1,
  'sparse': False,
  'multilabel': False,
  'multiclass': False,
  'target_type': 'classification',
  'signed': False})),
(0.120000, SimpleClassificationPipeline({'bal

## Model Run and Accuracy Stats

All in one place for easier comparison.

In [25]:
print("-----------Heart Disease---------------")
print(' ')
print(' ')

print("Model stats HD Holdout:")
print(automl_hd_ohe.sprint_statistics())
print(' ')
print("Accuracy score HD Holdout:")
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe))

print(' ')
print('-----------------------------------------')
print(' ')

print("Model stats HD Holdout Parallel:")
print(automl_hd_ohe_p.sprint_statistics())
print("Accuracy score HD Holdout Parallel:")
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe_p))

print(' ')
print('-----------------------------------------')
print(' ')

# #holdout parallel feat_type
# print("Model stats HD Holdout Feature Type Parallel:")
# print(automl_hd_ft_p.sprint_statistics())
# print(' ')
# print("Accuracy score HD Holdout Feature Type Parallel:")
# print(sklearn.metrics.accuracy_score(y_hd_test, \
#                                      predictions_hd_ft_p))

# print(' ')
# print('-----------------------------------------')
# print(' ')

# #cross validation parallel
# print("Model stats HD CV Parllel:")
# print(automl_hd_cv_p.sprint_statistics())
# print(' ')
# print("Accuracy score HD CV Parallel:")
# print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
#                                      predictions_hd_cv_p))

print(' ')
print(' ')

print("-----------Breast Cancer---------------")
print(' ')
print(' ')

print("Model stats BC Holdout Parallel:")
print(automl_bc.sprint_statistics())
print(' ')
print("Accuracy score BC Holdout Parallel:")
print(sklearn.metrics.accuracy_score(y_bc_test, \
                                     predictions_bc))

-----------Heart Disease---------------
 
 
Model stats HD Holdout:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.866667
  Number of target algorithm runs: 18
  Number of successful target algorithm runs: 18
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

 
Accuracy score HD Holdout:
0.75
 
-----------------------------------------
 
Model stats HD Holdout Parallel:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 25
  Number of successful target algorithm runs: 21
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

Accuracy score HD Holdout Parallel:
0.7368421052631579
 
----------------------------------

## Save Ensemble to Disk

In the specific case of [scikit-learn](https://scikit-learn.org/stable/modules/model_persistence.html#persistence-example), it may be better to use joblib’s replacement of pickle (dump & load), which is more efficient on objects that carry large numpy arrays internally as is often the case for fitted scikit-learn estimators, but can only pickle to the disk and not to a string.

In [26]:
from joblib import dump, load
dump(automl_bc, '../results/autosklearn_bc.joblib') 

['../results/autosklearn_bc.joblib']

## Save to Domino Stats File

To keep things simple, we pick one of the hd models. Saving stats to this file [allows Domino to track and trend them in the Experiment Manager](https://support.dominodatalab.com/hc/en-us/articles/204348169-Diagnostic-statistics-with-dominostats-json) when this notebook is run as a batch or scheduled job.

In [27]:
hd_acc = sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                        predictions_hd_ohe_p)
bc_acc = sklearn.metrics.accuracy_score(y_bc_test, \
                                        predictions_bc)

import json
with open('../dominostats.json', 'w') as f:
    f.write(json.dumps( {"HD_ACC": hd_acc, "BC_ACC": bc_acc}))